In [49]:
###利用selenium上传图像至花伴侣，并完成识别
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import win32gui
import win32con
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd
import pyautogui
import time
import pickle
import random
import glob
import pandas as pd

def uploads(filePath, browser_type="Chrome"):
    if browser_type == "Chrome":
        title = "打开"
    else:
        title = "文件上传"
    # 32770‐ComboBoxEx32 ‐ComboBox ‐Edit
    dialog = win32gui.FindWindow("#32770", title)  # 一级窗口 ‘打开窗口’
    # 二级
    ComboBoxEx32 = win32gui.FindWindowEx(dialog, 0, "ComboBoxEx32", None)
    # 三级
    ComboBox = win32gui.FindWindowEx(ComboBoxEx32, 0, "ComboBox", None)
    # 四级
    Edit = win32gui.FindWindowEx(ComboBox, 0, "Edit", None)
    # 32770‐Button
    button = win32gui.FindWindowEx(dialog, 0, "Button", None)  # 四级
    # 往文件名编辑框中输入文件路径
    # 上传操作
    win32gui.SendMessage(Edit, win32con.WM_SETTEXT, None, filePath)
    # 点击打开按钮
    win32gui.SendMessage(dialog, win32con.WM_COMMAND, 1, button)
    # 文件路径

if __name__ == '__main__':
    basepath = r"H:\test_rename"
    com = r"H:\测试集花伴侣测试.xlsx"
    base_url = "https://www.iplant.cn/"
    driver = webdriver.Chrome()

    driver.implicitly_wait(1)
    driver.maximize_window()
    driver.get(base_url)
    #upload_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "upfilediv")))
    #driver.find_element(By.CLASS_NAME,"upfilediv").click()
    #upload_input.send_keys(image_path)
    data = pd.read_excel(com)
    planf_list = data["Filename"].to_list()
    df = pd.DataFrame(columns=["Filename","PlantName","Result_url","top1_classname","top1_prob","top5_classname","top5_prob"])
    rest = [f for f in glob.glob(os.path.join(basepath,"*","*.jpg")) if f not in planf_list]
    for i,f in enumerate(rest):
        if f in planf_list:
            continue
        plantname = os.path.basename(f).split("_")[0]
        df.loc[i,"Filename"] = f
        df.loc[i,"PlantName"] = plantname
        try:
            el_file = driver.find_element(By.CLASS_NAME,"upfilediv")
            ActionChains(driver).move_to_element(el_file).click().perform()
            time.sleep(1)
            uploads(filePath=f)
            sleep(1)
            driver.refresh()
            current_url = driver.current_url
            df.loc[i,"Result_url"] = current_url
            driver.get(current_url)
            top = driver.find_elements(By.ID,"stugroup")
            category_info = top[0].text.split("\n")
            df.loc[i,"top1_classname"] = category_info[0]
            df.loc[i,"top1_prob"] = category_info[1]
            df.loc[i,"top5_classname"] = [category_info[j] for j in [0,2,4,6,8]]
            df.loc[i,"top5_prob"] = [category_info[j] for j in [1,3,5,7,9]]
        except Exception as e:
            print(e)

In [48]:
with pd.ExcelWriter(r"H:\测试集花伴侣测试1.xlsx") as writer:
    df.to_excel(writer)

,Filename,PlantName,Result_url,top1_classname,top1_prob,top5_classname,top5_prob
0,H:\test_rename\Androsace umbellata\Androsace u...,Androsace umbellata,https://www.iplant.cn/stuinfo/2364499,Androsace umbellata,95.23%,"[Androsace umbellata, Androsace septentrionali...","[95.23%, 1.07%, 0.81%, 0.64%, 0.58%]"
1,H:\test_rename\Androsace umbellata\Androsace u...,Androsace umbellata,https://www.iplant.cn/stuinfo/2364500,Androsace umbellata,95.55%,"[Androsace umbellata, Androsace maxima, Andros...","[95.55%, 1.92%, 1.21%, 0.19%, 0.19%]"
2,H:\test_rename\Androsace umbellata\Androsace u...,Androsace umbellata,https://www.iplant.cn/stuinfo/2364501,Androsace umbellata,19.47%,"[Androsace umbellata, Marchantia polymorpha, P...","[19.47%, 7.80%, 2.49%, 2.11%, 2.10%]"
3,H:\test_rename\Androsace umbellata\Androsace u...,Androsace umbellata,https://www.iplant.cn/stuinfo/2364502,Androsace umbellata,96.30%,"[Androsace umbellata, Primula merrilliana, And...","[96.30%, 0.50%, 0.40%, 0.30%, 0.21%]"
4,H:\test_rename\Androsace umbellata\Androsace u...,Androsace umbellata,https://www.iplant.cn/stuinfo/2364503,Androsace umbellata,99.54%,"[Androsace umbellata, Androsace maxima, Primul...","[99.54%, 0.18%, 0.08%, 0.03%, 0.02%]"
...,...,...,...,...,...,...,...
2588,H:\test_rename\Hippuris vulgaris\Hippuris vulg...,Hippuris vulgaris,NaN,NaN,NaN,NaN,NaN
2589,H:\test_rename\Hordeum jubatum\Hordeum jubatum...,Hordeum jubatum,NaN,NaN,NaN,NaN,NaN
2590,H:\test_rename\Hordeum jubatum\Hordeum jubatum...,Hordeum jubatum,NaN,NaN,NaN,NaN,NaN
2591,H:\test_rename\Hordeum jubatum\Hordeum jubatum...,Hordeum jubatum,NaN,NaN,NaN,NaN,NaN


In [98]:
###利用selenium上传图像至pl@antnet，并完成识别
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import win32gui
import win32con
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd
import pyautogui
import time
import pickle
import random
import glob
import pandas as pd

def uploads(filePath, browser_type="Chrome"):
    if browser_type == "Chrome":
        title = "打开"
    else:
        title = "文件上传"
    # 32770‐ComboBoxEx32 ‐ComboBox ‐Edit
    dialog = win32gui.FindWindow("#32770", title)  # 一级窗口 ‘打开窗口’
    # 二级
    ComboBoxEx32 = win32gui.FindWindowEx(dialog, 0, "ComboBoxEx32", None)
    # 三级
    ComboBox = win32gui.FindWindowEx(ComboBoxEx32, 0, "ComboBox", None)
    # 四级
    Edit = win32gui.FindWindowEx(ComboBox, 0, "Edit", None)
    # 32770‐Button
    button = win32gui.FindWindowEx(dialog, 0, "Button", None)  # 四级
    # 往文件名编辑框中输入文件路径
    # 上传操作
    win32gui.SendMessage(Edit, win32con.WM_SETTEXT, None, filePath)
    # 点击打开按钮
    win32gui.SendMessage(dialog, win32con.WM_COMMAND, 1, button)
    # 文件路径

if __name__ == '__main__':
    basepath = r"H:\test_rename"
    com = r"H:\测试集花伴侣测试.xlsx"
    base_url = "https://identify.plantnet.org/"
    driver = webdriver.Chrome()

    driver.implicitly_wait(1)
    driver.maximize_window()
    driver.get(base_url)
    #upload_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "upfilediv")))
    #driver.find_element(By.CLASS_NAME,"upfilediv").click()
    #upload_input.send_keys(image_path)
    #data = pd.read_excel(com)
    #planf_list = data["Filename"].to_list()
    df = pd.DataFrame(columns=["Filename","PlantName","top1_classname","top1_prob","top5_classname","top5_prob"])
    #rest = [f for f in glob.glob(os.path.join(basepath,"*","*.jpg")) if f not in planf_list]
    rest = glob.glob(os.path.join(basepath,"*","*.jpg"))[174:]
    for i,f in enumerate(rest):
        #if f in planf_list:
        #    continue
        plantname = os.path.basename(f).split("_")[0]
        df.loc[i,"Filename"] = f
        df.loc[i,"PlantName"] = plantname
        try:
            el_file = driver.find_element(By.CLASS_NAME,"custom-file-label")
            ActionChains(driver).move_to_element(el_file).click().perform()
            time.sleep(1)
            uploads(filePath=f)
            sleep(5)
            #driver.refresh()
            top = driver.find_elements(By.ID,"modal-results___BV_modal_body_")
            category_info = [j.split("\n") for j in top[0].text.split("%\n")]
            df.loc[i,"top1_classname"] = category_info[0][0]
            df.loc[i,"top1_prob"] = category_info[0][-1]
            df.loc[i,"top5_classname"] = [j[0] for j in category_info]
            df.loc[i,"top5_prob"] = [j[-1] for j in category_info]
        except Exception as e:
            print(e)
        finally:
            driver.refresh()

Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F6A813+48355]
	(No symbol) [0x00EFC4B1]
	(No symbol) [0x00E05358]
	(No symbol) [0x00DED293]
	(No symbol) [0x00E4E37B]
	(No symbol) [0x00E5C473]
	(No symbol) [0x00E4A536]
	(No symbol) [0x00E282DC]
	(No symbol) [0x00E293DD]
	GetHandleVerifier [0x011CAABD+2539405]
	GetHandleVerifier [0x0120A78F+2800735]
	GetHandleVerifier [0x0120456C+2775612]
	GetHandleVerifier [0x00FF51E0+616112]
	(No symbol) [0x00F05F8C]
	(No symbol) [0x00F02328]
	(No symbol) [0x00F0240B]
	(No symbol) [0x00EF4FF7]
	BaseThreadInitThunk [0x75AA00F9+25]
	RtlGetAppContainerNamedObjectPath [0x772D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x772D7B8E+238]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F6A813+48355]
	(No symbol) [0x00EFC4B1]
	(No symbol) [0x00E05358]
	(No symbol) [0x00DED293]
	(No symbol) [0x00E4E37B]
	(No symbol) [0x00E5C473]
	(No symbol) [0x00E4A536]
	(No symbol) [0x00E282DC]
	(No symbol) [0x00E293DD]
	GetHandleVerifier [0x011CAABD+2539405]
	GetHandleVerifier [0x0120A78F+2800735]
	GetHandleVerifier [0x0120456C+2775612]
	GetHandleVerifier [0x00FF51E0+616112]
	(No symbol) [0x00F05F8C]
	(No symbol) [0x00F02328]
	(No symbol) [0x00F0240B]
	(No symbol) [0x00EF4FF7]
	BaseThreadInitThunk [0x75AA00F9+25]
	RtlGetAppContainerNamedObjectPath [0x772D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x772D7B8E+238]


In [99]:
df

,Filename,PlantName,top1_classname,top1_prob,top5_classname,top5_prob
0,H:\test_rename\Adenophora tetraphylla\Adenopho...,Adenophora tetraphylla,Adenophora triphylla (Thunb.) A.DC.,39.88,"[Adenophora triphylla (Thunb.) A.DC., Isodon e...","[39.88, 0.72, 0.25, 0.15, 0.1%]"
1,H:\test_rename\Adenophora tetraphylla\Adenopho...,Adenophora tetraphylla,Adenophora triphylla (Thunb.) A.DC.,20.76,"[Adenophora triphylla (Thunb.) A.DC., Campanul...","[20.76, 3.08, 1.59, 1.57, 1.21%]"
2,H:\test_rename\Adenophora tetraphylla\Adenopho...,Adenophora tetraphylla,Loading...,Loading...,[Loading...],[Loading...]
3,H:\test_rename\Adenophora tetraphylla\Adenopho...,Adenophora tetraphylla,Adenophora triphylla (Thunb.) A.DC.,41.91,"[Adenophora triphylla (Thunb.) A.DC., Adenopho...","[41.91, 17.4, 1.21, 0.62, 0.54%]"
4,H:\test_rename\Adenophora tetraphylla\Adenopho...,Adenophora tetraphylla,Adenophora triphylla (Thunb.) A.DC.,33.29,"[Adenophora triphylla (Thunb.) A.DC., Adenopho...","[33.29, 8.64, 0.53, 0.48, 0.4%]"
...,...,...,...,...,...,...
89,H:\test_rename\Allium condensatum\Allium conde...,Allium condensatum,Tecoma fulva (Cav.) G.Don,5.94,"[Tecoma fulva (Cav.) G.Don, Argentina peduncul...","[5.94, 5.91, 4.84, 2.64, 2.38%]"
90,H:\test_rename\Allium condensatum\Allium conde...,Allium condensatum,Loading...,Loading...,[Loading...],[Loading...]
91,H:\test_rename\Allium condensatum\Allium conde...,Allium condensatum,Allium saxatile M.Bieb.,25.79,"[Allium saxatile M.Bieb., Allium cepa L., Alli...","[25.79, 4.05, 2.43, 0.73, 0.61%]"
92,H:\test_rename\Allium macrostemon\Allium macro...,Allium macrostemon,Thalictrum foliolosum DC.,17.79,"[Thalictrum foliolosum DC., Thalictrum aquileg...","[17.79, 3.04, 2.07, 1.35, 1.24%]"


In [100]:
with pd.ExcelWriter(r"H:\Pl@antnet测试(174-268).xlsx") as writer:
    df.to_excel(writer)

In [81]:
a = [i.split("\n") for i in top[0].text.split("%\n")]

In [86]:
[i[0] for i in a]

['Achillea distans Waldst. & Kit. ex Willd.',
 'Achillea × roseoalba Ehrend.',
 'Achillea millefolium L.',
 'Achillea asiatica Serg.',
 'Achillea nobilis L.']

In [ ]:
#调用pl@antnet API
import requests
import json
from pprint import pprint
if __name__ == '__main__':
    inpath = r"H:\test_rename"
    API_KEY = "2b10HmngCaic2byAvT5qb6DJu"  # Set you API_KEY here
    PROJECT = "all" # try "weurope" or "canada"
    api_endpoint = f"https://my-api.plantnet.org/v2/identify/{PROJECT}?api-key={API_KEY}"
    image_list = glob.glob(os.path.join(inpath,"*","*.jpg"))[:490]
    df = pd.DataFrame(columns=["Filename","fPlantName","plnet_top1_scientificName","plnet_top1_sciNameWithoutAuthor","plnet_top1_score","plnet_top5_scientificName","plnet_top5_sciNameWithoutAuthor","plnet_top5_score"])
    ###字段说明{Filename:输入图像的绝对路径，fPlantName：有文件名解析该图像鉴定物种名称，plnet_top1_scientificName：由pl@antnet识别的top1物种的科学名，plnet_top1_sciNameNotAuthor：无作者的植物名称（简名）}
    for i,f in enumerate(image_list):
        plantname = os.path.basename(f).split("_")[0]
        df.loc[i,"Filename"] = f
        df.loc[i,"fPlantName"] = plantname
        try:
            image_data = open(f, 'rb')
            files = [
                ('images', (f, image_data)),
            ]
            req = requests.Request('POST', url=api_endpoint, files=files)
            prepared = req.prepare()
            s = requests.Session()
            response = s.send(prepared)
            json_result = json.loads(response.text)
            result_dict = json_result["results"]
            if response.status_code == 200:
                df.loc[i,"plnet_top1_scientificName"] = result_dict[0]["species"]["scientificName"]
                df.loc[i,"plnet_top1_sciNameWithoutAuthor"] = result_dict[0]["species"]["scientificNameWithoutAuthor"]
                df.loc[i,"plnet_top1_score"] = result_dict[0]["score"]
                df.loc[i,"plnet_top5_scientificName"] = [result_dict[i]["species"]["scientificName"] for i in range(5)]
                df.loc[i,"plnet_top5_sciNameWithoutAuthor"] = [result_dict[i]["species"]["scientificNameWithoutAuthor"] for i in range(5)]
                df.loc[i,"plnet_top5_score"] = [result_dict[i]["score"] for i in range(5)]
        except Exception as e:
            print(e)
        finally:
            print(f)
    with pd.ExcelWriter(r"H:\pl@antnet_0-490.xlsx") as writer:
        df.to_excel(writer)

H:\test_rename\Achillea asiatica\Achillea asiatica_0.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_1.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_2.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_3.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_4.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_5.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_6.jpg
H:\test_rename\Achillea asiatica\Achillea asiatica_7.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_0.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_1.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_10.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_11.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_12.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_13.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_14.jpg
H:\test_rename\Achillea millefolium\Achillea millefolium_15.jpg
H:\test_rename\Achillea millefoliu

In [119]:
json_result["results"][0]["species"]

{'scientificNameWithoutAuthor': 'Achillea nobilis',
 'scientificNameAuthorship': 'L.',
 'genus': {'scientificNameWithoutAuthor': 'Achillea',
  'scientificNameAuthorship': '',
  'scientificName': 'Achillea'},
 'family': {'scientificNameWithoutAuthor': 'Compositae',
  'scientificNameAuthorship': '',
  'scientificName': 'Compositae'},
 'commonNames': ['Noble yarrow', 'Yarrow', 'Creamy Yarrow'],
 'scientificName': 'Achillea nobilis L.'}

In [7]:
#对花伴侣测试的结果进行解析，和实际进行比对，计算准确率
import os
import pandas as pd

if __name__ == '__main__':
    inpath = r"E:\论文\草地植物分类\测试\花伴侣测试"
    df = pd.read_excel(os.path.join(inpath,"花伴侣测试结果_去重后.xlsx"))
    rows,cols = df.shape
    for i in range(rows):
        df.loc[i,"top1_test_lable"] = 1 if df.loc[i,"PlantName"] == df.loc[i,"top1_classname"] else 0
        df.loc[i,"top5_test_lable"] = 1 if df.loc[i,"PlantName"] in df.loc[i,"top5_classname"] else 0
    with pd.ExcelWriter(os.path.join(inpath,"花伴侣测试结果_parser.xlsx")) as writer:
        df.to_excel(writer)

In [10]:
#对pl@antnet测试的结果进行解析，和实际进行比对，计算准确率
import os
import pandas as pd

if __name__ == '__main__':
    inpath = r"E:\论文\草地植物分类\测试\pl@antnet"
    df = pd.read_excel(os.path.join(inpath,"pl@antnet植物识别测试结果.xlsx"))
    rows,cols = df.shape
    for i in range(rows):
        try:
            df.loc[i,"top1_test_lable"] = 1 if df.loc[i,"fPlantName"] == df.loc[i,"plnet_top1_sciNameWithoutAuthor"] else 0
            df.loc[i,"top5_test_lable"] = 1 if df.loc[i,"fPlantName"] in df.loc[i,"plnet_top5_sciNameWithoutAuthor"] else 0
        except Exception as e:
            print(e)
    with pd.ExcelWriter(os.path.join(inpath,"pl@antnet_parser.xlsx")) as writer:
        df.to_excel(writer)

argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type

,Unnamed: 0.1,Filename,fPlantName,plnet_top1_scientificName,plnet_top1_sciNameWithoutAuthor,plnet_top1_score,plnet_top5_scientificName,plnet_top5_sciNameWithoutAuthor,plnet_top5_score,Unnamed: 0,top1_test_lable,top5_test_lable
0,0,H:\test_rename\Achillea asiatica\Achillea asia...,Achillea asiatica,Achillea nobilis L.,Achillea nobilis,0.44314,"['Achillea nobilis L.', 'Achillea millefolium ...","['Achillea nobilis', 'Achillea millefolium', '...","[0.44314, 0.13786, 0.11165, 0.06035, 0.02959]",0.0,0.0,0.0
1,1,H:\test_rename\Achillea asiatica\Achillea asia...,Achillea asiatica,Achillea setacea Waldst. & Kit.,Achillea setacea,0.40311,"['Achillea setacea Waldst. & Kit.', 'Achillea ...","['Achillea setacea', 'Achillea asiatica', 'Ach...","[0.40311, 0.15666, 0.06094, 0.02885, 0.01148]",1.0,0.0,1.0
2,2,H:\test_rename\Achillea asiatica\Achillea asia...,Achillea asiatica,Achillea asiatica Serg.,Achillea asiatica,0.31597,"['Achillea asiatica Serg.', 'Achillea alpina L...","['Achillea asiatica', 'Achillea alpina', 'Achi...","[0.31597, 0.13025, 0.126, 0.11359, 0.10838]",2.0,1.0,1.0
3,3,H:\test_rename\Achillea asiatica\Achillea asia...,Achillea asiatica,Achillea nobilis L.,Achillea nobilis,0.59993,"['Achillea nobilis L.', 'Achillea roseo-alba E...","['Achillea nobilis', 'Achillea roseo-alba', 'A...","[0.59993, 0.20365, 0.02212, 0.01438, 0.01128]",3.0,0.0,0.0
4,4,H:\test_rename\Achillea asiatica\Achillea asia...,Achillea asiatica,Achillea asiatica Serg.,Achillea asiatica,0.70474,"['Achillea asiatica Serg.', 'Achillea roseo-al...","['Achillea asiatica', 'Achillea roseo-alba', '...","[0.70474, 0.09425, 0.03361, 0.03204, 0.02715]",4.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7080,6790,H:\test_rename\Veronicastrum sibiricum\Veronic...,Veronicastrum sibiricum,Veronicastrum virginicum (L.) Farw.,Veronicastrum virginicum,0.59581,"['Veronicastrum virginicum (L.) Farw.', 'Veron...","['Veronicastrum virginicum', 'Veronicastrum si...","[0.59581, 0.3172, 0.05918, 0.00424, 0.00149]",NaN,NaN,NaN
7081,6827,H:\test_rename\Vicia ramuliflora\Vicia ramulif...,Vicia ramuliflora,Lathyrus vernus (L.) Bernh.,Lathyrus vernus,0.07047,"['Lathyrus vernus (L.) Bernh.', 'Buckleya dist...","['Lathyrus vernus', 'Buckleya distichophylla',...","[0.07047, 0.05074, 0.03648, 0.03538, 0.02032]",NaN,NaN,NaN
7082,7029,H:\test_rename\Viola variegata\Viola variegata...,Viola variegata,Cyclamen graecum Link,Cyclamen graecum,0.36309,"['Cyclamen graecum Link', 'Viola selkirkii Pur...","['Cyclamen graecum', 'Viola selkirkii', 'Cycla...","[0.36309, 0.06416, 0.0396, 0.01939, 0.01829]",NaN,NaN,NaN
7083,409,H:\test_rename\Androsace gmelinii\Androsace gm...,Androsace gmelinii,Isoetes duriei Bory,Isoetes duriei,0.10797,"['Isoetes duriei Bory', 'Plantago heterophylla...","['Isoetes duriei', 'Plantago heterophylla', 'L...","[0.10797, 0.0614, 0.03422, 0.01035, 0.00956]",NaN,NaN,NaN


In [7]:
##花伴侣识别结果按类统计
#将分类表和推理表进行匹配，方便后续分类进行统计
import pandas as pd
import matplotlib.pyplot as plt

if __name__ == '__main__':
    infer_table = r"E:\论文\草地植物分类\测试\pl@antnet\pl@antnet_parser.xlsx"
    infer_df = pd.read_excel(infer_table)
    infer_df_sum = infer_df.groupby(by=["fPlantName"])["top1_test_lable","top5_test_lable"].sum()     #统计每个种类top1和top5预测正确数量

C:\Users\kbqin\AppData\Local\Temp\ipykernel_20980\1993975708.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  infer_df_sum = infer_df.groupby(by=["fPlantName"])["top1_test_lable","top5_test_lable"].sum()     #统计每个种类top1和top5预测正确数量


In [9]:
infer_df_sum.loc[infer_df_sum.top1_test_lable > 0,:]

,top1_test_lable,top5_test_lable
fPlantName,,
Achillea asiatica,5.0,6.0
Achillea millefolium,2.0,9.0
Acorus calamus,3.0,3.0
Actaea dahurica,4.0,7.0
Aegopodium alpestre,1.0,2.0
...,...,...
Viola collina,8.0,11.0
Viola mirabilis,1.0,3.0
Woodsia ilvensis,1.0,2.0
